In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [3]:
data.head()
#displays the pandas dataframe

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data = np.array(data)
#converts data to numpy arrays
m, n = data.shape 
# m = no. of examples, n = amount of features + 1 (due to label column)
np.random.shuffle(data)

data_dev = data[0:1000].T # 0 to 1000 examples
Y_dev = data_dev[0]       # Y has the value of the digit
X_dev = data_dev[1:n]     # X has all the n features (784 pixels here)
X_dev = X_dev / 255.
 
data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [5]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5 #dimensions are written in braces, 10 rows for the 10 units in hidden layer and 784 connections of each unit of hidden layer, each connection having its own wt.
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

#randn generates random values between -0.5 to 0.5

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [6]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2
  

In [7]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[8 5 5 ... 8 3 0] [2 9 8 ... 6 2 2]
0.092
Iteration:  10
[0 5 0 ... 8 3 0] [2 9 8 ... 6 2 2]
0.13553658536585367
Iteration:  20
[0 5 0 ... 0 3 0] [2 9 8 ... 6 2 2]
0.16058536585365854
Iteration:  30
[0 5 0 ... 0 3 0] [2 9 8 ... 6 2 2]
0.1941219512195122
Iteration:  40
[0 9 0 ... 0 3 0] [2 9 8 ... 6 2 2]
0.24192682926829268
Iteration:  50
[0 9 0 ... 2 2 0] [2 9 8 ... 6 2 2]
0.2752926829268293
Iteration:  60
[0 9 3 ... 2 2 0] [2 9 8 ... 6 2 2]
0.2976341463414634
Iteration:  70
[0 9 3 ... 2 2 0] [2 9 8 ... 6 2 2]
0.3254390243902439
Iteration:  80
[6 9 3 ... 2 2 6] [2 9 8 ... 6 2 2]
0.3621219512195122
Iteration:  90
[6 9 3 ... 2 2 6] [2 9 8 ... 6 2 2]
0.4022439024390244
Iteration:  100
[6 9 3 ... 2 2 6] [2 9 8 ... 6 2 2]
0.43429268292682927
Iteration:  110
[6 9 3 ... 2 2 6] [2 9 8 ... 6 2 2]
0.46563414634146344
Iteration:  120
[2 9 3 ... 6 2 6] [2 9 8 ... 6 2 2]
0.49397560975609756
Iteration:  130
[2 9 3 ... 6 2 2] [2 9 8 ... 6 2 2]
0.5263170731707317
Iteration:  140
[2 9 3 .